In [ ]:
# Take a copy for comparison of filtered with unfiltered
filt = df.copy()
filt = filt.rolling(11).median()

# Normalise for outlier removal

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer

# scaler = PowerTransformer(method='yeo-johnson')

# ct = ColumnTransformer([('transform', scaler, input_columns)], remainder='passthrough')

# filt.dropna(inplace=True)
# trans = ct.fit_transform(filt)

# trans = pd.DataFrame(trans, columns=input_columns)

# Remove Outliers prior to feature engineering

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

# fig, ax = plt.subplots()
# ax.boxplot(trans)
# plt.show()

# Create velocity, impedance and porosity logs

In [14]:
df = rplog_convert(df, "DTC", "DTS")
df['PHIE'] = density_porosity(df.RHOB, 2.8, 1.1)
# Repeat for median filtered version
filt = rplog_convert(filt, "DTC", "DTS")
filt['PHIE'] = density_porosity(filt.RHOB, 2.8, 1.1)

# Calculate Shale logs from GR log

In [15]:
# Calculates a number of VSH versions
df = vshale_from_gr(df)
filt = vshale_from_gr(filt)

# Calculate Water Saturation
## Archies Equation

$$S_w =\Bigg( \frac{aR_w}{R_t \phi^m}\Bigg)^\frac{1}{n}$$
    
$S_w$ : water saturation of the uninvaded zone \
`n`  : saturation exponent, 1.8-4.0, usually 2.0 \
$R_w$ : formation water resistivity at formation temperature \
$\phi$: porosity \
`m`  : cementation exponent, 1.7-3.0, usually 2.0 \
$R_t$ : true resistivity of the formation, corrected for invasion \
`a`  : 1 (carbonate), 0.62 (sandstone) \

In [1]:
# Archies Equation: 
n = 2
rw = df.RSHA # RXO
phi = df.PHIE
m = 2
rt = df.RDEP # RMED
a = 0.62

# Sw = ((a * rw) / (rt * phi^m))**0.5

NameError: name 'df' is not defined

In [ ]:
# Remove DT, DTS, VSH_{versions}, IGR. Replaced with VP, VS
df = df[rock_physics_curves]
filt = filt[rock_physics_curves]

# Plot basic rock physics logs and compare unfiltered with filtered

In [ ]:
# Plot logs from unfiltered
fig, axs = plt.subplots(1, len(df.columns), figsize=(14, 10), sharey=True)
for ic, col in enumerate(df.columns):
    axs[ic].plot(df[col], df.index)
    axs[ic].set_xlabel(col)
axs[0].set_ylim(df.index.values[-1], df.index.values[0])

In [ ]:
# Plot logs from filtered
fig, axs = plt.subplots(1, len(filt.columns), figsize=(14, 10), sharey=True)
for ic, col in enumerate(filt.columns):
    axs[ic].plot(filt[col], filt.index)
    axs[ic].set_xlabel(col)
axs[0].set_ylim(filt.index.values[-1], filt.index.values[0])

In [ ]:
# Plot these, histogram and boxplots ???
filt_unfilt_diff(df, filt)

# Calculate lithofacies (litho-fluid-facies)
ref: https://github.com/seg/tutorials-2015/blob/master/1504_Seismic_petrophysics_1/Seismic_petrophysics_1.ipynb

In [ ]:
# ztop = 3300; zbot = df.DEPTH_MD.max()

# logs = df.loc[(df.DEPTH_MD>=ztop) & (df.DEPTH_MD<=zbot)]

# sand_cutoff = 0.20
# brine_sand = ((logs.VSH <= sand_cutoff) & (logs.SW >= 0.9))
# oil_sand = ((logs.VSH <= sand_cutoff) & (logs.SW < 0.9))
# shale = (logs.VSH > sand_cutoff)

# temp_lfc = np.zeros(np.shape(logs.VSH))
# temp_lfc[brine_sand.values] = 1    # LFC will be 1 when ssb (brine sand flag) is True
# temp_lfc[oil_sand.values] = 2      # LFC will be 2 when sso (oil sand flag) is True
# temp_lfc[shale.values] = 4         # LFC will be 4 when sh (shale flag) is True
# logs['LFC'] = temp_lfc             # Copy the temporary log temp_lfc into the DataFrame with name `LFC`

# # logs.to_csv('qsiwell2_lfc.csv',index=False) # save the data for use in Part 2

# Plot unfiltered and filtered logs

In [ ]:
# Unfiltered
plotlog(df, 3300, 3470, 0.3, 0.35)

In [ ]:
# Filtered
plotlog(filt, 3300, 3470, 0.5, 0.55)